## <span style="color:#FFC1C1; font-weight:bold">2. nn.Module</span>

####  신경망 구성

- 레이어(layer): 신경망의 핵심 데이터 구조로 하나 이상의 텐서를 입력받아 하나 이상의 텐서를 출력
- 모듈(module): 한 개 이상의 계층이 모여서 구성
- 모델(model): 한 개 이상의 모듈이 모여서 구성

#### `torch.nn` 패키지

주로 가중치(weights), 편향(bias)값들이 내부에서 자동으로 생성되는 레이어들을 사용할 때 사용합니다! (`weight`값들을 직접 선언 안함)

1. `nn.Linear` 계층 예제

In [1]:
import torch
import torch.nn as nn

In [2]:
input = torch.randn(128, 20)
print(input)

m = nn.Linear(20, 30)
print(m)

output = m(input)
print(output)
print(output.size())

tensor([[-0.0676, -0.0989, -0.0670,  ...,  0.8140,  0.7822, -1.2714],
        [ 0.7406, -0.0217, -1.1408,  ...,  0.5388,  1.3454, -0.6178],
        [ 1.8179,  0.3228,  0.2990,  ..., -0.4869, -0.1050,  1.1327],
        ...,
        [-1.1455, -0.1215, -0.1995,  ..., -1.8318, -1.3728,  0.2914],
        [ 0.0791, -0.5305, -0.6160,  ...,  1.4103, -0.7648, -0.9064],
        [ 0.5358,  0.0339,  0.1657,  ...,  2.8516,  1.4224, -1.0641]])
Linear(in_features=20, out_features=30, bias=True)
tensor([[ 4.0962e-01, -1.2111e-02,  4.3441e-01,  ...,  5.6932e-01,
          4.8634e-01, -8.9426e-05],
        [ 6.1350e-01,  1.2618e+00,  2.8604e-02,  ..., -1.6180e-01,
         -4.6309e-02, -3.4037e-01],
        [-1.3091e-01, -4.3149e-01, -2.3915e-02,  ..., -4.7574e-01,
          7.4747e-01, -3.1817e-02],
        ...,
        [ 3.0377e-01, -1.9558e-01,  6.9033e-01,  ...,  4.5507e-01,
          3.8493e-01,  7.3751e-01],
        [ 5.1184e-01, -7.9964e-01,  4.6262e-02,  ...,  2.8012e-01,
          5.9790e-01, -

2. `nn.Conv2d` 계층 예제

In [3]:
input = torch.randn(20, 16, 50, 100)
print(input.size())

torch.Size([20, 16, 50, 100])


In [4]:
m = nn.Conv2d(16, 33, 3, stride=2)
m = nn.Conv2d(16, 33, (3, 5), stride=(2,1), padding=(4,2))
m = nn.Conv2d(16, 33, (3, 5), stride=(2,1), padding=(4,2), dilation=(3,1))

In [5]:
output = m(input)
print(output.size())

torch.Size([20, 33, 26, 100])


#### `nn.Module` 상속 클래스 정의

nn.Module은 <span style="color:#FFC1C1; font-weight:bold">PyTorch의 모든 Neural Network의 Base Class</span>입니다.

이 클래스를 상속하여 사용자 정의 신경망 모델을 만들 수 있죠!

In [6]:
import torch
from torch import nn

### 기본 method  

간단하게 nn.Module을 상속한 Add 클래스를 만들어보겠습니다!

클래스 내에서 2가지를 반드시 정의해줘야합니다
- `__init__()`: 모델에서 사용될 모듈과 활성화 함수 등을 정의
- `forward()`: 모델에서 실행되어야 하는 연산을 정의

In [7]:
class Add(nn.Module):
    def __init__(self):
        super().__init__() # 반드시 Add class의 부모 클래스인 nn.Module을 super()을 사용해서 초기화 시켜줘야 한다.

    def forward(self, x1, x2):
        output = torch.add(x1, x2)
        return output

In [8]:
x1 = torch.tensor([1])
x2 = torch.tensor([2])

In [9]:
model = Add()
output = model(x1, x2)

In [10]:
print(output)

tensor([3])


엇... 그런데 우리가 흔히 알고 있는 클래스에서 함수 사용법이랑 다른거 눈치챘나요?

model.forward()와 같이 호출하지 않았는데 단순히 model 객체를 데이터와 함께 호출하면 자동으로 forward() 함수가 실행되었잖아요.

왜 이런 일이 발생하는지 이해하기 위해서 `nn.Module`의 소스코드를 뜯어봐야합니다!

https://github.com/pytorch/pytorch/blob/main/torch/nn/modules/module.py

그리고 이를 이해하기 전에 `__call__`이란 무엇을 의미하는지 짚고 넘어갈게요.

- `__call__` : 클래스의 인스턴스를 마치 함수처럼 호출할 수 있게 해주는 메소드

In [11]:
class Plus:

	def add(self, n1, n2):
		return n1 + n2

	__call__ = add

In [12]:
myinstance = Plus()
myinstance(1, 2)

3

즉, `__call__`가 `add` 메소드를 가리키고 있어서, Plus의 인스턴스를 함수처럼 호출했을때 add 메소드가 실행되는거에요!

그렇다면 `nn.Module`의 소스코드 상 `__call__`은 무엇을 어떤 메소드를 가리키고 있을까요? (1634번째 줄)

![__call__메소드](https://github.com/jkyoon2/ds_codingCamp/blob/main/04_pytorch/image/__call__.png?raw=true)

`_wrapped_call_impl`을 가리키고 있군요. (1507번째 줄)

`_wrapped_call_impl`은 무엇을 가리킬까요?

![_wrapped_call_impl](https://github.com/jkyoon2/ds_codingCamp/blob/main/04_pytorch/image/_wrapped_call_impl.png?raw=true)

`_call_impl`을 가리키고 있군요. (1513번째 줄)

`_call_impl`은 복잡한 error handling 코드를 가지고 있지만, 문제가 없다면 `forward`을 하는 것을 볼 수 있습니다!

![_call_impl](https://github.com/jkyoon2/ds_codingCamp/blob/main/04_pytorch/image/_call_impl.png?raw=true)

즉, 우리가 오버라이딩한 forward() 메소드는 클래스를 인스턴스 한 후, input만 넣어주면 실행되는 것이죠!!

여기까지 왔다면, 다된것과 다름 없습니다.

이제 추가적으로 많이 쓰이는 함수들을 살펴보겠습니다.

### 추가적 method

1. `apply`

- 모든 submodule에 함수를 적용하는 역할

잠깐! submodule을 짚고 넘어갈게요.  

우선, 모듈은 다른 모듈을 포함할 수 있고, 트리 구조로 형성됩니다. 예를 들어, nn.Sequential 안에는 nn.Linear, nn.Conv2d 포함될 수 있겠죠.

이때, 모델 내의 모든 nn.Module을 상속 받는 클래스는 submodule입니다.

apply는 모든 submodule에 재귀적(recursive)으로 연산을 수행하는 함수입니다. 파라미터를 설정할 때 모델에 사용하면, 모든 트리구조의 submodule들에 일괄 적용할 수 있겠죠~?


In [7]:
@torch.no_grad()
def init_weights(m):
     print(m)
     if type(m) == nn.Linear:   # 모델의 모든 submodule에 대해 nn.Linear가 있으면 아래를 수행
         m.weight.fill_(1.0)    # fill_(1.0)은 fill의 in-place operation으로, nn.Linear의 weight를 모두 1.0으로 채운다는 뜻
         print(m.weight)

In [15]:
net = nn.Sequential(nn.Linear(2, 2), nn.Linear(2, 2))
net.apply(init_weights)                    # apply(fn) 적용 방법

Linear(in_features=2, out_features=2, bias=True)
Parameter containing:
tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
Linear(in_features=2, out_features=2, bias=True)
Parameter containing:
tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
)


Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
)

2. `cpu`, `cuda`

- 모델을 어느 디바이스에 올릴 것인지 결정
- 기본적으로 모델은 CPU에 올라가 있음

In [1]:
import torch
import torch.nn as nn

In [2]:
torch.cuda.is_available()  # True인 경우 GPU, 즉 cuda를 사용할 수 있다는 뜻(cuda는 gpu로 학습하기 위해 사용하는 프로그램)

True

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

데이터가 클수록, 많을수록 cuda를 사용하면 시간을 많이 아낄 수 있습니다.😎

model을 cuda에 올리는 방법으로는 아래와 같이 '**to(device)**'를 붙이면 됩니다. cuda를 사용할 수 있는 상황에서는 cuda에서 모델을 불러오고, 그렇지 않으면 cpu를 활용합니다.

이때, 위와 같이 device를 꼭 먼저 정의해주어야합니다.

[예시] clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").**to(device)**

3. `parameters`

- 모델의 모든 파라미터를 담은 iterator를 return
- 보통 optimizer 선언할 시 argument로 넣어줌

In [ ]:
from torch import optim

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

4. `state_dict`

- 모델의 submodule을 dictionary 형태로 반환
- 모델 저장/로드할 때 씀

아래는 파이토치 공식 문서에 나와있는 예시 코드입니다.

모델의 구조를 세세히 이해하려고 하기보단 우선 state_dict에 어떤 값들이 저장되는지 살펴봅시다.

In [19]:
import torch.nn.functional as F
# 모델 정의
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 모델 초기화
model = TheModelClass()

# 옵티마이저 초기화
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 모델의 state_dict 출력
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# 옵티마이저의 state_dict 출력
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict:
conv1.weight 	 torch.Size([6, 3, 5, 5])
conv1.bias 	 torch.Size([6])
conv2.weight 	 torch.Size([16, 6, 5, 5])
conv2.bias 	 torch.Size([16])
fc1.weight 	 torch.Size([120, 400])
fc1.bias 	 torch.Size([120])
fc2.weight 	 torch.Size([84, 120])
fc2.bias 	 torch.Size([84])
fc3.weight 	 torch.Size([10, 84])
fc3.bias 	 torch.Size([10])
Optimizer's state_dict:
state 	 {}
param_groups 	 [{'lr': 0.001, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'maximize': False, 'foreach': None, 'differentiable': False, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}]


In [20]:
torch.save(model, 'full_model.pt')            # 1번
torch.save(model.state_dict(), 'model.pt')    # 2번

**[모델 전체를 저장하는 1번 코드]**


*   파라미터, 에포크 등 모든 정보 저장
*   나중에 학습을 이어서 하고 싶을 때
*   더 큰 용량 필요

**[모델의 state_dict만 저장하는 2번 코드]**


*   가중치, 편향 등 학습되는 변수에 대한 정보 저장
*   코드 상 모델이 구현되어 있을 때만!
*   작은 용량 사용





5. `train`, `eval`

- 모델 train을 시작할 때는 `model.train()`, evaluation을 시작 할 때는 `model.eval()`을 사용

🙋‍♀️ <span style="color:#D8BFD8; font-weight:bold"> 이렇게 train과 evaluation 시작할때 `train()`, `eval()` 메소드를 호출하는 이유가 무엇일까요? </span> 🙋‍♀️

1. Dropout이나 BatchNorm 같은 레이어들은 학습 시와 평가 시에 다르게 동작하도록 설계되어 있는데요, `train()`, `eval()` 메소드를 통해 모델 내 해당 레이어들을 일일이 모드 변경하지 않고 손쉽게 일괄 변경 시켜줍니다.

2. 단순한 모델은 output만으로 loss를 구할 수 있었습니다. (output과 target을 통해 loss를 구한 후, `backward()`, `optimizer.step()`을 해주면 되죠) 하지만 보다 복잡한 모델들은 output만으로 loss를 구할 수 없습니다. 예를 들어, Object detection model인 detectron2만 봐도 forward() 내부에 loss를 구하는 과정이 포함되어 있으며, 심지어 loss도 detector_losses, proposal_losses로 2개입니다. 모델을 사용하는 사람 입장에서는 사용자가 loss를 계산하기 너무 어렵죠. 그래서 `forward()` 내부에 Loss를 구하는 과정을 추상화하고, train mode이면 loss나 loss의 총합을 return하고 inference mode(train mode = False)이면 prediction을 return하는 경우가 많습니다.

참고를 위해 detectron2 `forward()` 내부에 구현된 loss 를 첨부합니다~

![detectron2_forward()](https://github.com/jkyoon2/ds_codingCamp/blob/main/04_pytorch/image/detectron2_forward().png?raw=true)

그럼 이제 모든 준비를 마친 셈입니다.

이제 직접... 사과 토마토 복숭아 분류 모델을 만들어봅시다 🍎🍅🍑